In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd
from fbprophet import Prophet
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")

In [17]:
#DATA PREPARATION FOR UNIVARIATE PROPHET MODEL

#GATHER ATHENS DATA FROM ALL AVAILABLE YEARS
Athens_Data = pd.read_csv('Athens_data.csv')

Athens_Data.rename(columns = {Athens_Data.columns[0]:'ds'}, inplace = True)
#KEEP ONLY THE POLLUTANTS COLUMNS
cols=['pm25', 'pm10', 'no2', 'o3', 'so2', 'ds']

#MAKE VALUES NUMERIC
Athens_Data_uni = Athens_Data.loc[:, Athens_Data.columns.intersection(cols)]
Athens_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']] = Athens_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']].apply(pd.to_numeric, errors='coerce')

#FILL NAN VALUES
Athens_Data_uni = Athens_Data_uni.iloc[1: , :]

#SOME VALUES ARE EMPTY, BUT NOT RECOGNIZED AS NAN. REPLACE EMPTY VALUES WITH NAN TO FILL THEM LATER
Athens_Data_uni = Athens_Data_uni.replace(r'^\s*$', np.nan, regex=True)
Athens_Data_uni = Athens_Data_uni.fillna(method='ffill')
Athens_Data_uni = Athens_Data_uni.fillna(method='bfill')

#REMOVE DUPLICATE VALUES
Athens_Data_uni = Athens_Data_uni[~Athens_Data_uni.index.duplicated(keep='first')]

#Fb prophet model predicts univariate time series. It requires that the Date index column is named 'ds'
#and the values column is named 'y'
Athens_Data_uni['ds'] = pd.DatetimeIndex(Athens_Data_uni['ds'])

In [3]:
Athens_Data_uni

,ds,no2,o3,pm10,pm25,so2
1,2019-11-08,0.423792,0.470135,0.442623,0.352381,0.039683
2,2019-11-09,0.442379,0.454721,0.213115,0.228571,0.039683
3,2019-11-10,0.237918,0.516378,0.131148,0.152381,0.039683
4,2019-11-11,0.356877,0.375723,0.229508,0.180952,0.039683
5,2019-11-12,0.490706,0.470135,0.459016,0.485714,0.039683
...,...,...,...,...,...,...
834,2022-03-15,0.371747,0.539499,0.327869,0.419048,0.277778
835,2022-03-16,0.442379,0.547206,0.426230,0.542857,0.317460
836,2022-03-17,0.338290,0.587669,0.459016,0.504762,0.198413
837,2022-03-18,0.204461,0.633911,0.295082,0.342857,0.079365


In [12]:
#Check data for stationarity to apply the ARIMA model.

#The null hypothesis is that the time series is non-stationary.
#Using the Augmented Dickey Fuller test (adfuller()) we can check for stationarity.
#If p-value < significance level(0.05), the we reject the null hypothesis.
#Otherwise, the time series is non-stationary and needs differencing.

print("Check if Athens_Data is stationary :")
for col in ['no2', 'o3', 'so2', 'pm10', 'pm25']:
    result = adfuller(Athens_Data[col])
    print('p-value for ' + col + ' : %f' % result[1])

Check if Athens_Data is stationary :
p-value for no2 : 0.000000
p-value for o3 : 0.185124
p-value for so2 : 0.000001
p-value for pm10 : 0.000000
p-value for pm25 : 0.000000


In [13]:
Athens_Data_uni['o3'] = Athens_Data_uni['o3'].diff()

In [11]:
def fbprophet_predict(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    
    #SPLIT TO TRAIN AND TEST PORTIONS
    train_size = 0.8
    train = time_series[:int(train_size*(len(time_series)))]
    test = time_series[int(train_size*(len(time_series))):]
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Athens[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Athens[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Athens[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [7]:
#FB PROPHET REQUIRES THE DATAFRAME TO HAVE TWO COLUMNS. 'ds' AND 'y', 'ds' IS THE TIME INDEX AND 'y' IS THE
#VALUE OF THE POLLUTANT

#RENAME COLUMNS TO FIT TO THE PROPHET MODEL
Athens_o3 = Athens_Data_uni[['ds', 'o3']]
Athens_o3.rename(columns = {'o3':'y'}, inplace = True)

Athens_no2 = Athens_Data_uni[['ds', 'no2']]
Athens_no2.rename(columns = {'no2':'y'}, inplace = True)

Athens_so2 = Athens_Data_uni[['ds', 'so2']]
Athens_so2.rename(columns = {'so2':'y'}, inplace = True)

Athens_pm10 = Athens_Data_uni[['ds', 'pm10']]
Athens_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Athens_pm25 = Athens_Data_uni[['ds', 'pm25']]
Athens_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Athens_o3, Athens_no2, Athens_so2, Athens_pm10, Athens_pm25]

In [12]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    
    cmp_month_test = pd.Series(test['y'])
    cmp_month_test.index = test['ds']
    cmp_month_test.index = pd.to_datetime(cmp_month_test.index)
    
    cmp_month_forecast = pd.Series(forecast['yhat'])
    cmp_month_forecast.index = forecast['ds']
    cmp_month_forecast.index = pd.to_datetime(cmp_month_forecast.index)
    
    print("Monthly prediction error for " + str(pollutants[i]) + ":\n")
    
    print("MAE: " + str(mean_absolute_error(cmp_month_test.resample("1m").mean().bfill(), cmp_month_forecast.resample("1m").mean().bfill())))
    print("MSE: " + str(mean_squared_error(cmp_month_test.resample("1m").mean().bfill(), cmp_month_forecast.resample("1m").mean().bfill())))
    print("RMSE: " + str(math.sqrt(mean_squared_error(cmp_month_test.resample("1m").mean().bfill(), cmp_month_forecast.resample("1m").mean().bfill()))))
    
    
    i = i+1

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-08  0.006752   -0.118591    0.135799
1 2021-09-09  0.006488   -0.122421    0.135483
2 2021-09-10 -0.012038   -0.148551    0.118825
3 2021-09-11  0.011291   -0.120855    0.147883
4 2021-09-12  0.020931   -0.108392    0.150635
          ds         y
0 2021-09-08 -0.055877
1 2021-09-09 -0.053950
2 2021-09-10  0.023121
3 2021-09-11  0.078998
4 2021-09-12  0.211946
FB Prophet RMSE for Athens[o3]: 0.10655574865383032

FB Prophet MSE for Athens[o3]: 0.011354127571178261

FB Prophet MAE for Athens[o3]: 0.07654105528978698

Monthly prediction error for o3:

MAE: 0.006416837806842581
MSE: 8.386756865368988e-05
RMSE: 0.009157923817857946


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-08  0.272681    0.055843    0.495806
1 2021-09-09  0.249364    0.033459    0.464855
2 2021-09-10  0.281400    0.057377    0.496656
3 2021-09-11  0.244117    0.049978    0.474118
4 2021-09-12  0.164482   -0.048877    0.368927
          ds         y
0 2021-09-08  0.118959
1 2021-09-09  0.167286
2 2021-09-10  0.286245
3 2021-09-11  0.390335
4 2021-09-12  0.442379
FB Prophet RMSE for Athens[no2]: 0.23506819389679431

FB Prophet MSE for Athens[no2]: 0.055257055781900884

FB Prophet MAE for Athens[no2]: 0.18983884663712397

Monthly prediction error for no2:

MAE: 0.1598463016602261
MSE: 0.02820978969049564
RMSE: 0.16795770208744712


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-08  0.299989    0.166452    0.430812
1 2021-09-09  0.295024    0.154917    0.435725
2 2021-09-10  0.302028    0.166546    0.459290
3 2021-09-11  0.288599    0.151355    0.423562
4 2021-09-12  0.277152    0.132586    0.417691
          ds         y
0 2021-09-08  0.198413
1 2021-09-09  0.238095
2 2021-09-10  0.238095
3 2021-09-11  0.119048
4 2021-09-12  0.198413
FB Prophet RMSE for Athens[so2]: 0.20969681691063943

FB Prophet MSE for Athens[so2]: 0.043972755022454235

FB Prophet MAE for Athens[so2]: 0.1716231431952845

Monthly prediction error for so2:

MAE: 0.14354143938181796
MSE: 0.023169368679963435
RMSE: 0.15221487667098585


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-08  0.398105    0.197299    0.605741
1 2021-09-09  0.367147    0.146384    0.550210
2 2021-09-10  0.357534    0.151130    0.566178
3 2021-09-11  0.386527    0.174592    0.605280
4 2021-09-12  0.384672    0.179459    0.595639
          ds         y
0 2021-09-08  0.344262
1 2021-09-09  0.131148
2 2021-09-10  0.229508
3 2021-09-11  0.295082
4 2021-09-12  0.426230
FB Prophet RMSE for Athens[pm10]: 0.17229507034669883

FB Prophet MSE for Athens[pm10]: 0.0296855912657739

FB Prophet MAE for Athens[pm10]: 0.14431306937199231

Monthly prediction error for pm10:

MAE: 0.10126587758752967
MSE: 0.014514658431369388
RMSE: 0.12047679623632672
          ds      yhat  yhat_lower  yhat_upper
0 2021-09-08  0.388271    0.213926    0.552838
1 2021-09-09  0.364057    0.202517    0.513031
2 2021-09-10  0.357505    0.187997    0.516262
3 2021-09-11  0.387461    0.228457    0.550709
4 2021-09-12  0.397610    0.232577    0.556970
          ds         y


### Seasonal Split

The seasonal decomposition of the Athens data we can see that there is a seasonality in the monthly resampled data. Specifically, for each pollutant the seasonal length of the data is:
- NO2: 12 months or 365 days
- SO2: 12 months
- O3: 12 months
- PM25: 12 months
- PM10: 12 months

In [32]:
def fbprophet_predict_seasonal(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    time_series.index = time_series['ds']
    #SPLIT TO TRAIN AND TEST PORTIONS
    train, test = time_series['2019-12-01' : '2021-12-31'], time_series['2022-01-01' : '2022-03-19']
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Athens[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Athens[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Athens[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [33]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict_seasonal(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.019042   -0.102151    0.154818
1 2022-01-02  0.032278   -0.103586    0.164359
2 2022-01-03 -0.041402   -0.176195    0.084627
3 2022-01-04  0.024674   -0.114109    0.158289
4 2022-01-05  0.001809   -0.133258    0.124186
          ds         y
0 2022-01-01 -0.025048
1 2022-01-02  0.025048
2 2022-01-03 -0.086705
3 2022-01-04  0.007707
4 2022-01-05  0.023121
FB Prophet RMSE for Athens[o3]: 0.08872714335120163

FB Prophet MSE for Athens[o3]: 0.007872505967264683

FB Prophet MAE for Athens[o3]: 0.0633491214286754



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.258724    0.070010    0.443696
1 2022-01-02  0.173422   -0.024563    0.355565
2 2022-01-03  0.281028    0.069335    0.474793
3 2022-01-04  0.257073    0.060377    0.472754
4 2022-01-05  0.268405    0.071889    0.470138
          ds         y
0 2022-01-01  0.371747
1 2022-01-02  0.390335
2 2022-01-03  0.390335
3 2022-01-04  0.542751
4 2022-01-05  0.457249
FB Prophet RMSE for Athens[no2]: 0.17956375436287084

FB Prophet MSE for Athens[no2]: 0.03224314188088941

FB Prophet MAE for Athens[no2]: 0.14931931141316923



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.190860    0.080272    0.305004
1 2022-01-02  0.186756    0.067862    0.303461
2 2022-01-03  0.205011    0.085878    0.329611
3 2022-01-04  0.213668    0.100008    0.323477
4 2022-01-05  0.228052    0.109735    0.344159
          ds         y
0 2022-01-01  0.277778
1 2022-01-02  0.238095
2 2022-01-03  0.357143
3 2022-01-04  0.357143
4 2022-01-05  0.277778
FB Prophet RMSE for Athens[so2]: 0.2601037654632151

FB Prophet MSE for Athens[so2]: 0.06765396880814323

FB Prophet MAE for Athens[so2]: 0.18102444746684235



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.357631    0.168898    0.548657
1 2022-01-02  0.352460    0.151980    0.540742
2 2022-01-03  0.361046    0.162866    0.554237
3 2022-01-04  0.356808    0.164690    0.534746
4 2022-01-05  0.363930    0.194395    0.565263
          ds         y
0 2022-01-01  0.327869
1 2022-01-02  0.983607
2 2022-01-03  0.311475
3 2022-01-04  0.377049
4 2022-01-05  0.311475
FB Prophet RMSE for Athens[pm10]: 0.1506783571630481

FB Prophet MSE for Athens[pm10]: 0.022703967317355085

FB Prophet MAE for Athens[pm10]: 0.11878638608831715

          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.366194    0.218700    0.529783
1 2022-01-02  0.377968    0.228885    0.534914
2 2022-01-03  0.365926    0.223723    0.524590
3 2022-01-04  0.355070    0.187682    0.513749
4 2022-01-05  0.367035    0.218219    0.524784
          ds         y
0 2022-01-01  0.457143
1 2022-01-02  1.000000
2 2022-01-03  0.419048
3 2022-01-04  0.400000
4 2022-01-05  0.4000

## Weekly predictions

In [9]:
Athens_Data_uni.index = Athens_Data_uni['ds']
Athens_Data_uni = Athens_Data_uni.drop(['ds'],axis=1)

Athens_Data_uni.index = pd.to_datetime(Athens_Data_uni.index)
Athens_weekly = Athens_Data_uni.resample("1w").mean().bfill()

Athens_weekly['ds'] = Athens_weekly.index

In [14]:
Athens_o3 = Athens_weekly[['ds', 'o3']]
Athens_o3.rename(columns = {'o3':'y'}, inplace = True)

Athens_no2 = Athens_weekly[['ds', 'no2']]
Athens_no2.rename(columns = {'no2':'y'}, inplace = True)

Athens_so2 = Athens_weekly[['ds', 'so2']]
Athens_so2.rename(columns = {'so2':'y'}, inplace = True)

Athens_pm10 = Athens_weekly[['ds', 'pm10']]
Athens_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Athens_pm25 = Athens_weekly[['ds', 'pm25']]
Athens_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Athens_o3, Athens_no2, Athens_so2, Athens_pm10, Athens_pm25]

In [16]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-10-03  0.642692    0.445673    0.803835
1 2021-10-10  0.645333    0.454585    0.839380
2 2021-10-17  0.647974    0.461168    0.834738
3 2021-10-24  0.650615    0.473449    0.828031
4 2021-10-31  0.653257    0.470406    0.837163
          ds         y
0 2021-10-03  0.617947
1 2021-10-10  0.549775
2 2021-10-17  0.549775
3 2021-10-24  0.549775
4 2021-10-31  0.549775
FB Prophet RMSE for Athens[o3]: 0.2594474582465087

FB Prophet MSE for Athens[o3]: 0.06731298359057387

FB Prophet MAE for Athens[o3]: 0.24001408577544545



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-10-03  0.280281    0.147675    0.427254
1 2021-10-10  0.279448    0.134191    0.417768
2 2021-10-17  0.278616    0.136199    0.419388
3 2021-10-24  0.277783    0.138415    0.416656
4 2021-10-31  0.276950    0.136902    0.418175
          ds         y
0 2021-10-03  0.319172
1 2021-10-10  0.106568
2 2021-10-17  0.106568
3 2021-10-24  0.106568
4 2021-10-31  0.106568
FB Prophet RMSE for Athens[no2]: 0.1451023231385573

FB Prophet MSE for Athens[no2]: 0.021054684180206305

FB Prophet MAE for Athens[no2]: 0.12344911792753066



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-10-03  0.258915    0.136926    0.363308
1 2021-10-10  0.259518    0.132717    0.364568
2 2021-10-17  0.260121    0.146740    0.373797
3 2021-10-24  0.260723    0.149339    0.381029
4 2021-10-31  0.261326    0.143654    0.374284
          ds         y
0 2021-10-03  0.221088
1 2021-10-10  0.211640
2 2021-10-17  0.211640
3 2021-10-24  0.211640
4 2021-10-31  0.211640
FB Prophet RMSE for Athens[so2]: 0.1478233638698948

FB Prophet MSE for Athens[so2]: 0.021851746905811324

FB Prophet MAE for Athens[so2]: 0.10513650059255748



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-10-03  0.355341    0.217943    0.495823
1 2021-10-10  0.356511    0.212117    0.490199
2 2021-10-17  0.357682    0.220315    0.490269
3 2021-10-24  0.358852    0.224651    0.490140
4 2021-10-31  0.360023    0.224502    0.502336
          ds         y
0 2021-10-03  0.283372
1 2021-10-10  0.273224
2 2021-10-17  0.273224
3 2021-10-24  0.273224
4 2021-10-31  0.273224
FB Prophet RMSE for Athens[pm10]: 0.099020291691642

FB Prophet MSE for Athens[pm10]: 0.009805018166697866

FB Prophet MAE for Athens[pm10]: 0.08543674825272332

          ds      yhat  yhat_lower  yhat_upper
0 2021-10-03  0.350201    0.239253    0.463943
1 2021-10-10  0.350652    0.234568    0.462474
2 2021-10-17  0.351102    0.237798    0.456152
3 2021-10-24  0.351552    0.248502    0.461002
4 2021-10-31  0.352003    0.242428    0.464931
          ds         y
0 2021-10-03  0.288435
1 2021-10-10  0.339683
2 2021-10-17  0.339683
3 2021-10-24  0.339683
4 2021-10-31  0.33968

## Monthly predictions

In [18]:
Athens_Data_uni.index = Athens_Data_uni['ds']
Athens_Data_uni = Athens_Data_uni.drop(['ds'],axis=1)

Athens_Data_uni.index = pd.to_datetime(Athens_Data_uni.index)
Athens_monthly = Athens_Data_uni.resample("1m").mean().bfill()

Athens_monthly['ds'] = Athens_monthly.index

In [19]:
Athens_o3 = Athens_monthly[['ds', 'o3']]
Athens_o3.rename(columns = {'o3':'y'}, inplace = True)

Athens_no2 = Athens_monthly[['ds', 'no2']]
Athens_no2.rename(columns = {'no2':'y'}, inplace = True)

Athens_so2 = Athens_monthly[['ds', 'so2']]
Athens_so2.rename(columns = {'so2':'y'}, inplace = True)

Athens_pm10 = Athens_monthly[['ds', 'pm10']]
Athens_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Athens_pm25 = Athens_monthly[['ds', 'pm25']]
Athens_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Athens_o3, Athens_no2, Athens_so2, Athens_pm10, Athens_pm25]

In [20]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 17.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 17.


          ds      yhat  yhat_lower  yhat_upper
0 2021-10-31  0.645920    0.479668    0.815957
1 2021-11-30  0.657046    0.487800    0.825614
2 2021-12-31  0.668544    0.493721    0.832765
3 2022-01-31  0.680041    0.518126    0.853923
4 2022-02-28  0.690426    0.520171    0.857108
          ds         y
0 2021-10-31  0.572254
1 2021-11-30  0.370841
2 2021-12-31  0.351545
3 2022-01-31  0.371807
4 2022-02-28  0.436072
FB Prophet RMSE for Athens[o3]: 0.25131108765679233

FB Prophet MSE for Athens[o3]: 0.06315726277923996

FB Prophet MAE for Athens[o3]: 0.23611478258041255



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 17.


          ds      yhat  yhat_lower  yhat_upper
0 2021-10-31  0.284147    0.198661    0.364354
1 2021-11-30  0.280976    0.203357    0.359089
2 2021-12-31  0.277700    0.197597    0.358696
3 2022-01-31  0.274423    0.192111    0.354771
4 2022-02-28  0.271464    0.189594    0.350731
          ds         y
0 2021-10-31  0.126394
1 2021-11-30  0.420818
2 2021-12-31  0.367910
3 2022-01-31  0.400648
4 2022-02-28  0.414100
FB Prophet RMSE for Athens[no2]: 0.12171389101541691

FB Prophet MSE for Athens[no2]: 0.014814271266112783

FB Prophet MAE for Athens[no2]: 0.11033418686558243



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 17.


          ds      yhat  yhat_lower  yhat_upper
0 2021-10-31  0.259817    0.158882    0.363924
1 2021-11-30  0.262445    0.163919    0.360820
2 2021-12-31  0.265160    0.166417    0.371688
3 2022-01-31  0.267875    0.166554    0.364365
4 2022-02-28  0.270328    0.171508    0.367326
          ds         y
0 2021-10-31  0.218254
1 2021-11-30  0.186508
2 2021-12-31  0.160010
3 2022-01-31  0.476446
4 2022-02-28  0.162982
FB Prophet RMSE for Athens[so2]: 0.12449972360462801

FB Prophet MSE for Athens[so2]: 0.01550018117762877

FB Prophet MAE for Athens[so2]: 0.11299052869365594



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 17.


          ds      yhat  yhat_lower  yhat_upper
0 2021-10-31  0.358222    0.288730    0.426862
1 2021-11-30  0.363240    0.295720    0.430857
2 2021-12-31  0.368425    0.301701    0.431668
3 2022-01-31  0.373610    0.301618    0.438778
4 2022-02-28  0.378293    0.306830    0.441152
          ds         y
0 2021-10-31  0.254098
1 2021-11-30  0.400546
2 2021-12-31  0.270756
3 2022-01-31  0.297726
4 2022-02-28  0.323185
FB Prophet RMSE for Athens[pm10]: 0.0874522921497539

FB Prophet MSE for Athens[pm10]: 0.007647903402245909

FB Prophet MAE for Athens[pm10]: 0.08231047377006005

          ds      yhat  yhat_lower  yhat_upper
0 2021-10-31  0.347897    0.279191    0.416474
1 2021-11-30  0.349363    0.289081    0.412818
2 2021-12-31  0.350878    0.282360    0.419480
3 2022-01-31  0.352394    0.282185    0.418693
4 2022-02-28  0.353762    0.285282    0.420466
          ds         y
0 2021-10-31  0.276190
1 2021-11-30  0.417778
2 2021-12-31  0.316743
3 2022-01-31  0.381260
4 2022-02-28  0.3687